In [2]:
import pandas as pd
from nba_api.stats.static import teams
# get_teams returns a list of 30 dictionaries, each an NBA team.

import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
from pathlib import Path

import time as time

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotInteractableException 
def clean_df(df,team):
    df = df.drop(columns = ['team'])
    drop = [c for c in df.columns if '%' in c]
    df = df.drop(columns = drop)
    new_df = pd.DataFrame()
    final = df.sum()
    final['team'] = team

    new_df= pd.DataFrame(final)

    new_df = new_df.T
    return new_df
    
def get_tables(url_base):
    xpath = '//*[@id="__next"]/div[2]/div[2]/div[3]/section[2]/div/div[2]/div[3]/table'
    data = []
    chrome_options = webdriver.ChromeOptions()
  
    prefs = {"profile.managed_default_content_settings.images": 2}
    chrome_options.add_experimental_option("prefs", prefs)
    driver = webdriver.Chrome(chrome_options=chrome_options)
    nba_teams = teams.get_teams()
    names = [i['abbreviation'] for i in nba_teams]
    tags = [i['id'] for i in nba_teams]   
    url_list = [url_base+str(tag) for tag in tags]
    for i in range(len(url_list)):
        url = url_list[i]
        team = names[i]
        print(url)
        
        driver.get(url)
        element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, xpath)))
        # Wait for the page to fully load
        driver.implicitly_wait(20)
        '''if check_exists_by_xpath(driver, "//a[contains(text(),'>')]/preceding-sibling::a[1]"):
            number_of_pages = int(driver.find_element(By.XPATH, "//a[contains(text(),'>')]/preceding-sibling::a[1]").text)
            print(number_of_pages)'''
        # Step 2: Parse HTML code and grab tables with Beautiful Soup
        soup = BeautifulSoup(driver.page_source, 'lxml')

        tables = soup.find_all('table')

        # Step 3: Read tables with Pandas read_html()
        dfs = pd.read_html(str(tables))
        #print(dfs)

        #print(f'Total tables: {len(dfs)}')
        #print(dfs[2].head())
    
        
        #return dfs
        df= dfs[-1]
        df.columns = [x.lower() for x in df.columns]
        
        df = clean_df(df,team)
        #drop = ['Unnamed: 16_level_1', 'Unnamed: 17_level_1', 'Unnamed: 18_level_1']
        #df.columns = df.columns.droplevel()
        #df = df.drop(columns = drop)
        data.append(df)
        time.sleep(2)
    driver.close()
    return pd.concat(data)

In [3]:
#terms = ['playtype/passing.csv','playtype/cs.csv','playtype/pullup.csv','playtype/oreb.csv','playtype/dreb.csv','playtype/elbow.csv','playtype/drives.csv']
terms = ['playtype/drives.csv']
terms = [t[:-4]+'_against.csv' for t in terms]




passing = 'https://www.nba.com/stats/teams/passing?PerMode=Totals'

cs ='https://www.nba.com/stats/teams/catch-shoot?PerMode=Totals'
pullup ='https://www.nba.com/stats/teams/pullup?PerMode=Totals'

oreb = 'https://www.nba.com/stats/teams/offensive-rebounding?PerMode=Totals'
dreb = 'https://www.nba.com/stats/teams/defensive-rebounding?PerMode=Totals'
elbow = 'https://www.nba.com/stats/teams/elbow-touch?PerMode=Totals'
drives = 'https://www.nba.com/stats/teams/drives?PerMode=Totals'
#url_list = [url1,url2,url3,url4,url5]
#url_list=[passing,cs,pullup,oreb,dreb,elbow,drives]
url_list=[drives]
def get_multi(url_list,terms):
    frames = []
    for i in range(len(url_list)):
        url = url_list[i]
        url+='&OpponentTeamID='
        df = get_tables(url)
        cols = df.columns.tolist()
        cols = cols[-1:] + cols[:-1]
        df = df[cols]
        frames.append(df)

    
        df.to_csv(terms[i],index = False)
    return frames

get_multi(url_list,terms)

<ipython-input-2-5bf653f6ede8>:37: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_options=chrome_options)


https://www.nba.com/stats/teams/drives?PerMode=Totals&OpponentTeamID=1610612737
https://www.nba.com/stats/teams/drives?PerMode=Totals&OpponentTeamID=1610612738
https://www.nba.com/stats/teams/drives?PerMode=Totals&OpponentTeamID=1610612739
https://www.nba.com/stats/teams/drives?PerMode=Totals&OpponentTeamID=1610612740
https://www.nba.com/stats/teams/drives?PerMode=Totals&OpponentTeamID=1610612741
https://www.nba.com/stats/teams/drives?PerMode=Totals&OpponentTeamID=1610612742
https://www.nba.com/stats/teams/drives?PerMode=Totals&OpponentTeamID=1610612743
https://www.nba.com/stats/teams/drives?PerMode=Totals&OpponentTeamID=1610612744
https://www.nba.com/stats/teams/drives?PerMode=Totals&OpponentTeamID=1610612745
https://www.nba.com/stats/teams/drives?PerMode=Totals&OpponentTeamID=1610612746
https://www.nba.com/stats/teams/drives?PerMode=Totals&OpponentTeamID=1610612747
https://www.nba.com/stats/teams/drives?PerMode=Totals&OpponentTeamID=1610612748
https://www.nba.com/stats/teams/drives?P

[  team  gp   w   l    min drives  fgm   fga  ftm  fta   pts  pass  ast   to  \
 0  ATL  54  27  27  13085   2662  574  1218  346  431  1537   960  264  196   
 0  BOS  53  16  37  12895   2460  612  1269  270  329  1539   807  200  160   
 0  CLE  55  22  33  13375   2789  605  1251  273  344  1527  1086  268  208   
 0  NOP  55  27  28  13350   2323  524  1044  328  409  1424   841  221  165   
 0  CHI  51  27  24  12365   2371  529  1043  286  363  1387   925  253  153   
 0  DAL  54  26  28  13135   2689  623  1167  342  428  1637  1072  283  178   
 0  DEN  54  17  37  13010   2503  618  1129  292  375  1579   962  237  157   
 0  GSW  52  26  26  12605   2522  516  1091  348  416  1410   990  246  154   
 0  HOU  53  40  13  12770   2184  471   925  266  332  1241   856  198  157   
 0  LAC  56  26  30  13515   2699  550  1175  334  441  1469  1085  257  145   
 0  LAL  54  29  25  13135   2494  600  1216  301  371  1538   894  230  151   
 0  MIA  54  25  29  13035   2095  466  

In [6]:
df = pd.read_csv('https://raw.githubusercontent.com/gabriel1200/site_Data/main/epm/epm.csv')

In [7]:
df

,player,gp,mpg,usg,off,def,epm,ew,ts%,efg,...,3pt_per,ft%_per,or%_per,dr%_per,a%_per,to%_per,st%_per,bl%_per,pos,team
0,Nikola Jokic,46,33.62,27,7.80,0.78,8.51,12.10,71.0,66.0,...,72.0,68.0,78.0,99.0,99.0,15.0,81.0,63.0,C,DEN
1,Luka Doncic,47,35.82,38,7.70,0.26,7.90,12.21,62.0,57.0,...,47.0,29.0,37.0,94.0,99.0,56.0,85.0,49.0,PG,DAL
2,Joel Embiid,40,34.71,38,5.70,1.69,7.30,9.60,64.0,56.0,...,51.0,80.0,74.0,96.0,84.0,50.0,63.0,91.0,C,PHI
3,Damian Lillard,41,36.21,32,8.01,-1.03,7.10,10.20,65.0,57.0,...,62.0,96.0,22.0,28.0,96.0,58.0,30.0,23.0,PG,POR
4,Stephen Curry,38,34.61,30,6.90,0.16,7.00,8.90,68.0,62.0,...,94.0,97.0,20.0,72.0,92.0,44.0,46.0,36.0,PG,GSW
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
489,Vince Williams Jr.,9,5.45,11,-4.50,-0.40,-4.90,-0.10,27.0,27.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SG,MEM
490,Peyton Watson,9,3.23,19,-3.90,-1.20,-5.10,0.00,42.0,42.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SG,DEN
491,Tyrese Martin,12,3.64,18,-3.60,-1.70,-5.30,-0.10,44.0,41.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SF,ATL
492,Udonis Haslem,6,7.65,12,-5.20,-0.20,-5.50,-0.10,12.0,8.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C,MIA


In [ ]:
df = get_tables('https://www.nba.com/stats/teams/drives?PerMode=Totals&OpponentTeamID=')

cols = df.columns.tolist()
cols = cols[-1:] + cols[:-1]
df = df[cols]
df.to_csv('playtype/drives_against.csv',index = False)

In [ ]:
test = frames[0]

In [ ]:
frames

In [ ]:
'''
    chrome_options = webdriver.ChromeOptions()
  
    prefs = {"profile.managed_default_content_settings.images": 2}
    chrome_options.add_experimental_option("prefs", prefs)

    driver = webdriver.Chrome(chrome_options=chrome_options)
'''